In [ ]:
import PyPDF2
from PyPDF2 import PdfFileReader, PdfFileWriter
from pdf2image import convert_from_path
import PIL
from PIL import Image, ImageFilter
from wand.image import Image as Img
import os
import os.path
from PIL import Image
import pyocr
import pyocr.builders
from PIL import Image
import subprocess
import pandas as pd
tools = pyocr.get_available_tools()[0]
# The tools are returned in the recommended order of usage

langs = tools.get_available_languages()
lang = langs[0]


In [ ]:
#pageObj = pdfReader.getPage(0) 

In [ ]:
print(pageObj.cropBox.getLowerLeft())
print(pageObj.cropBox.getUpperLeft())
print(pageObj.cropBox.getUpperRight())
print(pageObj.cropBox.getLowerRight())

In [ ]:
72*(2.84)

In [ ]:
## Got the first row
## need to turn image into pixels

In [ ]:
import pandas as pd

getAnswers("filled_koa-ca-1.pdf")

In [ ]:
###############################################################################
# Main logic for making directories of student answers
# The code takes in the packet of scanned answers and then 
# sorts the worksheets based on worksheet name and student name
#
# Student answers are then placed into their own subdirectory
###############################################################################

def getAnswers(pdfPacketName):
    
    sheet_id = []; name_id = []
    pdfFileObj = open(pdfPacketName, 'rb') 
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj) 
    print(pdfReader.numPages)
        
    for i in range(pdfReader.numPages):
        pageObj = pdfReader.getPage(i)

        pdfToPNG(pageObj, "identifier", (29, 740), (31 + 65, 745 + 26))
        
        
        ###############################################################################
        # This line of code processes the worksheet name
        # It takes the PNG rectangle box, lightens the 
        # rectangle box around the sheet name, and then 
        # darkens the actual characters
        # S.L. 7/18/19
        ###############################################################################
        bashCommand = """convert identifier.png -type Grayscale -negate -define 
        morphology:compose=darken -morphology Thinning 'Rectangle:1x80+0+0<' -negate 
        out.png""" # output file
        process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
        output, error = process.communicate()
        ###############################################################################

        worksheetName = tools.image_to_string(Image.open('out.png'), 
                                     builder=pyocr.builders.DigitBuilder())

        # Create worksheet directory if it doesn't exist
        if not os.path.exists(worksheetName):
            os.makedirs(worksheetName)
            
        pageObj = pdfReader.getPage(i)
        writer = PdfFileWriter()
        
        pdfToPNG(pageObj, "student", (200, 739), (282, 775))

        studentName = tools.image_to_string(Image.open('student.png'), 
                                     builder=pyocr.builders.DigitBuilder())
        print(studentName)
        name_id.append(studentName)
        sheet_id.append(worksheetName)
        pathName = worksheetName + "/" + studentName
        print(pathName)

        if not os.path.exists(pathName):
            os.makedirs(pathName)

        df = pd.read_csv('points.csv')

        #df['Koa-CA-1'].str.split(';')
        #for row in df.itertuples(index=True, name='Pandas'):
        #    print getattr(row, "c1"), getattr(row, "c2")
        #print(df)
        matrix = [list(l) for l in zip(*df.values)]

        getAttribute(pageObj, pathName, worksheetName, matrix)



In [ ]:
#################################################################
# outputName is the filename minus extension
# Usage: pdfToJPG(pageObj, identifier, (x, y), (x, y))
# this will create crop the pdf and creater identifier.jpg at
# the given coordinates
# TODO: pdfToPNG is a kind of inefficient function to keep calling
# A future improvement would just be converting to PNG once and then
# cropping, instead of converting pdf to png for each crop box
# S.L. 7/18/19
#################################################################

def pdfToPNG(pageObj, fileName, bottomLeft, upperRight):

    writer = PdfFileWriter()

    pageObj.cropBox.lowerLeft = bottomLeft
    pageObj.cropBox.upperRight = upperRight

    writer.addPage(pageObj)
    outstream = open(fileName + ".pdf", 'wb')
    writer.write(outstream)
    outstream.close()
    with Img(filename= fileName + ".pdf", resolution=300) as img:   # convert pdf to jpg
        img.compression_quality = 99 
        img.save(filename= fileName + ".png")
    img = Image.open(fileName + '.png')
    crop_img = img.crop((bottomLeft[0] / 72 * 300, (11 - (upperRight[1] / 72)) * 300, upperRight[0] / 72 * 300, (11 - (bottomLeft[1] / 72)) * 300))
    crop_img.save(fileName + '.png')
    os.remove(fileName + ".pdf")
    #img = Image.open(fileName + ".png")
    #img = img.convert('L')
    #img = img.crop
    #img.save(fileName + ".png")

In [ ]:
df = pd.DataFrame(sheet_id,  name_id)
df.columns = ['sheet_id']
df['student_id'] = df.index
df.to_csv('students_work.csv')

In [ ]:
df = pd.read_csv('points.csv')

#df['Koa-CA-1'].str.split(';')
#for row in df.itertuples(index=True, name='Pandas'):
#    print getattr(row, "c1"), getattr(row, "c2")
#print(df)
matrix = [list(l) for l in zip(*df.values)]

getAttribute(matrix[1])

In [ ]:
#################################################################
# getAttribute takes in a pageobject, pathname, and a dataframe
# and writes the individual student answers in the corresponding
# directory. Note: reliability depends on reading the correct
# worksheet name and student ID. 
# S.L. 7/18/19
#################################################################

def getAttribute(pageObj, pathName, worksheetName, matrix):
    
    df = pd.read_csv('points.csv')
    
    matrix = [list(l) for l in zip(*df.values)]
    try:
        pointsList = df[worksheetName]
    except:
        print("Error: The CSV file does not contain points that correspond to the worksheet name - " + worksheetName)
        print("This can occur when the program misreads the worksheet name from the PDF file.")
        print("Note - This error means that an invalid directory has also been created in the top level.")
        pathname = input("""Please enter the correct pathname in the format \(Worksheet Name\/Student Name if known\), or
        type 'quit' to stop the kernel""")
        
        if pathname == 'quit':
            sys.exit(1)
        else:
            os.makedirs(pathName)
    colIndex = df.columns.get_loc(worksheetName)
     
    for i in range(len(pointsList)):
        
        bottomLeft = getCorner(pointsList[i])
        #print(bottomLeft)
        topRight = getSides(bottomLeft, pointsList[i])
        #print(topRight)
        pdfToPNG(pageObj, pathName + "/" + matrix[colIndex - 1][i], bottomLeft, topRight)
        print(matrix[colIndex - 1][i])
        

In [ ]:
#def getAnswer(pathname, bottomLeft, topRight):
    

In [ ]:
def getCorner(string): 
    #print(string[3:19])
    bl = string.split("; size-")[0].split("BL-")[1].split(",")
    return (((float(bl[0][1:]))) * 72, (11 - float(bl[1][:len(bl[1]) - 1])) * 72)

In [ ]:
def getSides(bottomLeft, string):
    string = string.split("; size-")[1].split(",")
    #print(string)
    return((bottomLeft[0] + 72 * float(string[0][1:])), (bottomLeft[1] + 72 * float(string[1][:len(string[1]) - 1])))
    
    

In [ ]:
def imageprepare(argv):
    
    im = Image.open(argv).convert('L')
    width = float(im.size[0])
    height = float(im.size[1])
    newImage = Image.new('L' , (28,28), (255)) ## Create white canvas of 28x28
    
    
    if width > height:
        nheight = int(round((20.0 / width * height), 0))
        if (nheight == 0):
            nheight = 1
            
        img = im.resize((20, nheight), Image.ANTIALIAS).filter(ImageFilter.SHARPEN)
        wtop = int(round(((28 - nheight)/ 2),0))
        newImage.paste(img, (4, wtop))
    else:   
        nwidth = int(round((20.0 / height * width), 0))
        if(nwidth == 0):
            nwidth = 1
        img = im.resize((nwidth,20),Image.ANTIALIAS).filter(ImageFilter.SHARPEN)
        wleft = int(round(((28 - nwidth) / 2), 0))
        newImage.paste(img, (wleft,4))

    tv = list(newImage.getdata())
    
    tva = [(255 - x)*1.0 / 255.0 for x in tv]
  
    return tva

In [ ]:
d = imageprepare('A.png')

In [ ]:
https://towardsdatascience.com/image-classification-in-10-minutes-with-mnist-dataset-54c35b77a38d
import tensorflow as tf
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [ ]:
    """    
    for i in range(questions, 0, -1):
        for j in range(parts):
            if j not in answers[i - 1]:
                continue
            else:

                fileName = "image_crop_" + "Q" + str(i) + "_P" + str(j)
                #print(fileName)

                leftSide = sideMargin + (horizantalSpacing * j) + (72 * j)
                bottomSide = bottomMargin + ((5 - i) * 72) + ((5 - i) * verticalSpacing)

                pageObj.cropBox.lowerLeft = (leftSide , bottomSide)
                pageObj.cropBox.upperRight = (leftSide + 72 , bottomSide + 72)
                writer.addPage(pageObj)
                outstream = open(fileName + ".pdf",'wb')
                writer.write(outstream)
                outstream.close()
                with Img(filename= fileName + ".pdf", resolution=300) as img:   # convert pdf to jpg
                    img.compression_quality = 99 
                    img.save(filename= fileName + ".jpg")
                    os.remove(fileName + ".pdf")  #remove the pdf file
    """